In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify

import tensorflow as tf
import keras.backend as K
from keras.models import load_model

from skimage.morphology import skeletonize
from skan import Skeleton, summarize
from skan.csr import skeleton_to_csgraph
from skan import draw
import skan
from skimage.morphology import skeletonize
import networkx as nx

from data.data_processing import *
from utils.helpers import *
from utils.metrics import *
from features.features import *

In [2]:
# Change if more/less then 5 plants in petri dish
# number_of_plants = int(input('Enter the number of plants in the petri dish: '))
number_of_plants = 5

In [3]:
# Create all required folders if not existent
create_folder('plants')
create_folder('input')

Folder 'plants' already exists.
Folder 'input' already exists.


In [4]:
# Load the models
root_segmentation_model = tf.keras.models.load_model(
    'model_refrence/root_4_ss.h5', custom_objects={'f1': f1, 'iou': iou})
occlusion_inpainter = tf.keras.models.load_model(
    'model_refrence/occlusion_rs_1_256.h5', custom_objects={'f1': f1, 'iou': iou}, compile=False)
occlusion_inpainter.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=[f1, iou])
shoot_segmentation_model = tf.keras.models.load_model(
    'model_refrence/shoot_exp_1.h5', custom_objects={'f1': f1, 'iou': iou})

In [5]:
# Save the model predictions (each image has its own sub-folder)
save_prediction_for_folder('input', root_segmentation_model, occlusion_inpainter,
                           shoot_segmentation_model, padder, 'plants', verbose=True)

2024-05-15 11:15:52,245 - features.features - INFO - Saving predictions for images in the input...
2024-05-15 11:15:52,793 - features.features - INFO - Loaded image: 006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected.png with shape: (3006, 4202)
2024-05-15 11:15:52,793 - features.features - INFO - Creating masks...
2024-05-15 11:16:08,100 - features.features - INFO - Created root mask...
2024-05-15 11:16:11,227 - features.features - INFO - Created shoot mask...
2024-05-15 11:16:11,227 - features.features - INFO - Fixing occlusion in the mask...
2024-05-15 11:16:11,684 - features.features - INFO - Refinement step: 0 / 10
2024-05-15 11:16:13,463 - features.features - INFO - Refinement step: 1 / 10
2024-05-15 11:16:14,934 - features.features - INFO - Refinement step: 2 / 10
2024-05-15 11:16:16,367 - features.features - INFO - Refinement step: 3 / 10
2024-05-15 11:16:17,800 - features.features - INFO - Refinement step: 4 / 10
2024-05-15 11:16:19,251 - features.features - INFO -

Folder 'plants\006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected' already exists.


2024-05-15 11:16:28,687 - features.features - INFO - Original image saved for: 006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected.png
2024-05-15 11:16:28,755 - features.features - INFO - Root mask saved for: 006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected.png
2024-05-15 11:16:28,810 - features.features - INFO - Shoot mask saved for: 006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected.png
2024-05-15 11:16:28,872 - features.features - INFO - Occlusion mask saved for: 006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected.png
2024-05-15 11:16:28,944 - features.features - INFO - Full root mask saved for: 006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected.png


In [6]:
# Create an image to overlay the root and shoot masks (saves the image in the same folder)
overlay_masks_on_image('plants')

2024-05-15 11:16:28,977 - features.features - INFO - Overlaying masks on images in the plants...


This has to be changed either way so no reason to reformat!

In [7]:
# Iterate through each subfolder in the 'plants' folder
for subdir, dirs, files in os.walk('plants'):
    for file in files:
        # Check if the file is a root mask image
        if file.endswith("_root_mask.png"):
            root_mask_path = os.path.join(subdir, file)
            # Read the mask image in grayscale
            root_mask = cv2.imread(root_mask_path, cv2.IMREAD_GRAYSCALE)

            # Connected components labeling on the root mask
            retval, labels, stats, centroids = cv2.connectedComponentsWithStats(
                root_mask)

            # Create a mask to retain only regions with area above the threshold
            mask_values = [255 if (stats[label, cv2.CC_STAT_TOP] < 800 and
                                   stats[label, cv2.CC_STAT_TOP] > 300) else 0
                           for label in range(1, retval)]

            # Apply the mask values to create a new mask
            mask = np.zeros_like(root_mask)
            for label, value in zip(range(1, retval), mask_values):
                mask[labels == label] = value

            # Apply the mask to the original root mask to get the final image
            final_image = cv2.bitwise_and(root_mask, root_mask, mask=mask)
            final_image = final_image[:, 850:3500]
            # Extract and save ROIs based on the final image
            for i in range(number_of_plants):
                start_col = i * final_image.shape[1] // number_of_plants
                end_col = (i + 1) * final_image.shape[1] // number_of_plants
                roi = final_image[:, start_col:end_col]
                # Save each ROI with a sequential number
                roi_filename = f"roi_{i+1}.png"  # Naming each ROI sequentially
                roi_path = os.path.join(subdir, roi_filename)
                if np.count_nonzero(roi) != 0:
                    skeletonized_roi = skeletonize(roi)
                    skeleton_graph = skan.summarize(Skeleton(skeletonized_roi))
                    num_skeletons = len(skeleton_graph['skeleton-id'].unique())
                    cv2.imwrite(roi_path, roi)
                else:
                    empty_roi = np.zeros_like(roi)
                    cv2.imwrite(roi_path, empty_roi)

In [8]:
for folder in os.listdir('plants'):
    measure_images_in_folder('plants/' + folder)

2024-05-15 11:16:37,143 - features.features - INFO - Measuring images in the plants/006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected...
2024-05-15 11:16:37,156 - features.features - INFO - Processing image: 006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected_occlusion_mask.png...
2024-05-15 11:16:37,157 - features.features - INFO - Processing image: 006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected_original_padded.png...
2024-05-15 11:16:37,157 - features.features - INFO - Processing image: 006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected_overlayed.png...
2024-05-15 11:16:37,157 - features.features - INFO - Processing image: 006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected_root_mask.png...
2024-05-15 11:16:37,157 - features.features - INFO - Processing image: 006_43-17-ROOT1-2023-08-08_mock_pH5_f6h1_02-Fish Eye Corrected_root_mask_inpainted.png...
2024-05-15 11:16:37,165 - features.features - INFO - Processing image: 006

: 